<h1>Parsing<h1>

Суммарно парсила сайт 2 дня. Финальный парсинг занял 4 часа.
Но давайте начнем с самого начала.
Первым делом я пошла на сайт https://www.kaggle.com/datasets/joebeachcapital/ign-games/data?select=ign.csv и
просто забрала оттуда датасет
Далее мне нужно было его обогатить, поскольку там данные только про игры до 2016 года.
Я отправилась на сайт https://www.ign.com/reviews.

In [1]:
import requests
from fake_useragent import UserAgent
import time
import pandas as pd

In [2]:
# С сайта https://selenium-python.readthedocs.io/getting-started.html#simple-usage
# на всякий случай предварительно качаю всё
from selenium import webdriver
from selenium.webdriver.common.by import By

Давайте начнем с самого начала.
Первым делом я пошла на сайт https://www.kaggle.com/datasets/joebeachcapital/ign-games/data?select=ign.csv и
просто забрала оттуда датасет
Далее мне нужно было его обогатить, поскольку там данные только про игры до 2016 года.
Я отправилась на сайт https://www.ign.com/reviews.

In [6]:
def find_u_gen():
    with open("../All_data_files/ign.csv", "r", encoding="utf-8") as ign:
        df = pd.read_csv(ign)
        unique_genres = list(df["genre"].unique())
        uni_gen = [x for x in unique_genres if isinstance(x, str) and ", " not in x]
    return uni_gen

In [7]:
def find_u_pltf():
    with open("../All_data_files/ign.csv", "r", encoding="utf-8") as ign:
        df = pd.read_csv(ign)
        unique_pltf = list(df["platform"].unique())
        uni_pl = [x for x in unique_pltf if isinstance(x, str)]
    return uni_pl

In [11]:
unique_genres = find_u_gen()
unique_platforms = find_u_pltf()
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
}
url = "https://www.ign.com/reviews"

**Далее идет кусок кода, который я делала на Pycharm. Весь парсинг занял много часов, поэтому когда я переносила файлы в Jupiter, я решила, что этот кусок нет смысла прогонять заново**

In [ ]:
# для того, чтобы начать парсинг, я буду использовать модуль fake_useragent.
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
print(response.status_code)

 Я решила парсить сайт со всеми обзорами, а не только игровыми,
 так как страница с игровыми обзорами не парсилась дальше 17-ой страницы.
Я не смогла найти решение, поэтому пошла через главную страницу.
Через fake_useragent получается обмануть сайт, тут не пришлось выдумывать каких-то сложных масок.

In [ ]:
# однако тут всплывает новая проблема. Сайт который я хочу парсить - динамический. То есть он прогружается при скролинге
# Это очень печально, потому что тут даже нельзя пощелкать страницы, чтобы найти их значения. Но я не отчаиваюсь
# и иду в интернет искать, что делать. Мне повезло - на сайте IGN нет явной пагинации, спасибо...
# Принято решение использовать селениум
# Устанавливаю
# Нужно настроить связь питона и браузера, поэтому:
#************************************
options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={UserAgent().random}")
driver = webdriver.Chrome(options=options)
driver.get(url)

 Далее я столкнулась с проблемой - сайт не имеет явных страниц, его надо прокручивать.
 Немного поборовшись с ним, получилось сделать то, что мне нужно.
Но возникла новая проблема - сайт парсится долго, а сколько конкретно мне нужно прокруток я не знаю.
Поставила 600 наугад, потому что вручную прокрутить и проверить было бы еще сложнее
В итоге я смогла собрать данные по всем недостающим мне годам
(получилось, что дата обзора не совпадала с датой выхода, что, наверное, должно было быть очевидно,
так что у меня появилось 5352 новых ссылок, которые покрыли вообще все годы с 1970 до 2024)

так как у меня динамический сайт, делаю вот так: https://digitalsfera.ru/prokrutka-stranitsy-v-selenium-python
я взяла код отсюда, добавила ограничение на количество страниц,
также я добавила в конец скрипт для записи всех ссылок в отдельный файл ign_links_more.txt

In [ ]:
last_height = driver.execute_script("return document.body.scrollHeight")
pages = [i for i in range(20)]
n = 0
while n < 600:
    # Прокрутка вниз
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Пауза, пока загрузится страница.
    # Вычисляем новую высоту прокрутки и сравниваем с последней высотой прокрутки.
    time.sleep(3)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    n += 1
    print(n)
    print(last_height, new_height)
    last_height = new_height

In [ ]:
with open("ign_links_more.txt", "w", encoding="utf-8") as ign:
    # найдем все ссылки на каждую страницу и сложим их в новый файл
    start_info = driver.find_elements(By.CLASS_NAME, "item-body")
    for string in start_info:
        href = string.get_attribute("href")
        ign.write(href + "\n")